# Imports

In [1]:
import pandas as pd
import json

# Get the bounding boxes

In [2]:
bounding_boxes = {}
with open("./bounding_boxes.json") as json_file:
    bounding_boxes = json.load(json_file)

# Clean FIRM

### Read

In [3]:
df = pd.read_csv("../../data/fire_archive_M6_157443.csv")
df["Location"] = None

### Round lat long and get rid of duplicates

In [4]:
df_round = df
df_round["latlong"] = df["latitude"].map(lambda x: str(round(x, 1))) + ", " + df["longitude"].map(lambda x: str(round(x, 1)))
df_round = df_round.drop_duplicates(subset=["latlong"])
df_round

,latitude,longitude,brightness,acq_date,confidence,bright_t31,type,Location,latlong
0,-15.8931,136.6094,324.4,1/01/2015,28,302.3,0,None,"-15.9, 136.6"
1,-14.9764,145.2801,320.6,1/01/2015,24,294.8,0,None,"-15.0, 145.3"
2,-18.5115,139.5995,331.8,1/01/2015,37,305.0,0,None,"-18.5, 139.6"
3,-17.1160,122.2857,315.7,1/01/2015,0,294.9,0,None,"-17.1, 122.3"
4,-18.0797,122.6967,313.7,1/01/2015,31,292.9,0,None,"-18.1, 122.7"
...,...,...,...,...,...,...,...,...,...
671877,-20.6312,119.8370,344.4,31/12/2017,81,322.3,0,None,"-20.6, 119.8"
671878,-16.7236,129.4765,327.4,31/12/2017,65,295.4,0,None,"-16.7, 129.5"
671895,-22.4574,134.5212,312.0,31/12/2017,84,297.2,0,None,"-22.5, 134.5"
671953,-18.3129,134.4838,304.4,31/12/2017,48,277.7,0,None,"-18.3, 134.5"


### Put city names in FIRM where lat long fall in a bounding box

In [5]:
for i, r in df_round.iterrows():
    # lat = r["latitude"]
    # lon = r["longitude"]

    if i % 1000 == 0:
        print(i)

    for location, box in bounding_boxes.items():
        # print(box, i, location)
        # print(lat, lon, lat < box[1], lat > box[0], lon > box[2], lon < box[3])
        if r["latitude"] < box[1] and r["latitude"] > box[0] and r["longitude"] > box[2] and r["longitude"] < box[3]:
            df_round.loc[i, 'Location'] = location
            break
        else:
            continue

0
9000
12000
16000
18000
36000
41000
42000
54000
57000
65000
70000
74000
84000
113000
114000
117000
142000
171000
175000
180000
189000
201000
204000
209000
236000
244000
263000
267000
330000
348000
379000
384000
399000
468000
491000
556000
588000
627000
635000
636000
652000


In [6]:
df_chosen = df_round.dropna()
df_chosen

,latitude,longitude,brightness,acq_date,confidence,bright_t31,type,Location,latlong
0,-15.8931,136.6094,324.4,1/01/2015,28,302.3,0,test,"-15.9, 136.6"
29,-31.9887,115.9483,326.2,1/01/2015,58,307.0,2,Perth,"-32.0, 115.9"
30,-32.0663,115.9202,325.5,1/01/2015,45,307.7,2,Perth,"-32.1, 115.9"
31,-31.8603,115.9087,324.5,1/01/2015,49,307.4,2,Perth,"-31.9, 115.9"
32,-31.7952,115.8318,324.1,1/01/2015,35,307.7,2,Perth,"-31.8, 115.8"
...,...,...,...,...,...,...,...,...,...
665699,-19.7681,142.9142,334.1,22/12/2017,81,310.4,0,Richmond,"-19.8, 142.9"
665712,-19.4853,142.2803,337.5,22/12/2017,86,306.2,0,Richmond,"-19.5, 142.3"
666070,-19.4831,142.2489,370.1,22/12/2017,100,314.6,0,Richmond,"-19.5, 142.2"
668357,-37.7084,145.0999,327.8,25/12/2017,80,305.9,2,Melbourne,"-37.7, 145.1"


### Write the file

In [7]:
df_chosen.to_csv("../../data/chosen_FIRN.csv")

# Group and count 

In [8]:
clean_FIRM_df = df_chosen
clean_FIRM_df["count"] = 1
clean_FIRM_df = clean_FIRM_df.groupby(['Location', 'acq_date']).agg({'count': 'size', 'latitude': 'mean', 'longitude': 'mean', 'brightness': 'mean', 'bright_t31': 'mean', 'confidence': 'mean'}).reset_index()

clean_FIRM_df.to_csv("../../data/chosen_FIRM_clean.csv")
clean_FIRM_df

,Location,acq_date,count,latitude,longitude,brightness,bright_t31,confidence
0,Adelaide,12/05/2015,1,-35.1873,138.6786,313.7,281.8,74.0
1,Adelaide,15/03/2015,1,-35.0109,138.5761,316.1,297.9,58.0
2,Adelaide,15/04/2015,1,-34.6313,138.8095,353.0,302.6,96.0
3,Adelaide,2/01/2015,1,-34.7548,138.7946,410.6,328.2,100.0
4,Adelaide,22/04/2015,1,-35.0766,138.7239,301.4,286.2,37.0
...,...,...,...,...,...,...,...,...
331,Wollongong,24/05/2017,1,-34.5751,150.7834,301.7,289.4,48.0
332,Wollongong,25/11/2016,1,-34.3663,150.9096,309.1,297.7,0.0
333,Wollongong,4/01/2015,1,-34.4583,150.8844,326.8,304.2,60.0
334,Wollongong,4/11/2016,1,-34.4257,150.8099,326.5,298.3,82.0


# Merge

### Read weather data

In [9]:
weather_df = pd.read_csv("../../data/weatherAUS.csv")

### Perform the merge based on location + date

In [17]:
merged_df = weather_df.merge(clean_FIRM_df, left_on=["Location", "Date"], right_on=["Location", "acq_date"], how="left")
merged_df

,Date,Location,MinTemp,MaxTemp,AvgTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure,Temp9am,Temp3pm,acq_date,count,latitude,longitude,brightness,bright_t31,confidence
0,1/01/2015,Albury,11.4,33.5,22.45,0.0,WSW,30,ESE,W,...,1012.25,21.0,32.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2/01/2015,Albury,15.5,39.6,27.55,0.0,NE,56,ESE,ESE,...,1014.20,25.6,38.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3/01/2015,Albury,17.1,38.3,27.7,0.0,NNE,48,NE,N,...,1015.10,29.2,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/01/2015,Albury,26.0,33.1,29.55,0.0,NNE,41,ESE,W,...,1012.85,27.4,30.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5/01/2015,Albury,19.0,35.2,27.1,0.0,E,33,SSE,SE,...,1016.05,25.6,32.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38531,29/12/2016,Hobart,2.6,17.7,13.25,10.4,SE,33,SSE,ESE,...,1020.75,17.2,33.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38532,30/12/2016,Hobart,3.2,19.5,14.3,4.4,W,35,SE,NE,...,1020.55,15.2,34.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38533,31/12/2016,Hobart,5.7,20.1,8.55,1.0,WNW,41,NE,NNW,...,1023.50,17.9,36.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38534,1/01/2017,Hobart,12.8,23.7,10.15,11.0,WNW,57,NE,NW,...,1021.00,15.9,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Write to CSV

In [18]:
merged_df.to_csv("../../data/final_ML_data.csv")